In [39]:
from random import choices, randint, randrange, random
from typing import List, Optional, Callable, Tuple
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt
from functools import partial


In [40]:
Genome = List[int]
Population = List[Genome]
FitnessFunc = Callable[[Genome], float]
PopulateFunc = Callable[[], Population]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]

Schewel

In [41]:
# n_point = 100
# k_max = 250
# dim = 3
# epsilon = 10**(-7)
# def objective_function(x):
#     f=0
#     for i in range (len(x)):
#         sum_sq = 0
#         for j in range (i+1):
#             sum_sq += x[j]
#         f += sum_sq**2 
#     return f
# boundaries = np.array([(-5,5) for _ in range (dim)])

Problem 1 - Paper Pak Kun

In [42]:
n_point = 50
k_max = 250
dim = 2
epsilon = 10**(-7)

def objective_function(x):
    f1 = np.exp(x[0]-x[1])-np.sin(x[0]+x[1])
    f2 = (x[0]*x[1])**2-np.cos(x[0]+x[1])
    f_list = [f1,f2]
    denom = 0
    for f in f_list:
        denom +=np.abs(f)
    F = 1/(1+denom)
    return F
boundaries = np.array([(-10,10) for _ in range (dim)])

Saddle Note Bifurcation

In [43]:
# n_point = 20
# k_max = 250
# dim = 2
# epsilon = 10**(-7)

# def objective_function(X):
#     f1 = X[0]+X[1]**2 #-2*x
#     f_list = [f1]
#     denom = 1
#     for f in f_list:
#         denom +=np.abs(f)
#     F = 1/denom
#     return F

# boundaries = np.array([(-10,10) for _ in range (dim)])

In [44]:
min_value = boundaries.min()
max_value = boundaries.max()
num_bits = 64  # Number of bits for each number

In [45]:
"""GENERATE POINTS USING SOBOL SEQUENCE"""
def generate_points(dim,npoint,low=-10,high=10):
    if type(low) != type(high):
        raise TypeError('The type of "low" and "high" should be the same.')
    if type(low) == int:
        boundaries = [(low,high) for _ in range (dim)]
    elif type(low) == list or type(low) == np.ndarray:
        if len(low) != len(high):
            raise TypeError('The length of "low" and "high" should be the same.')
        else:
            boundaries = [(low[i],high[i]) for i in range (len(low))]

    # Generate Sobol sequence points
    sobol_points = sobol_seq.i4_sobol_generate(dim, npoint)

    # Scale the Sobol points to fit within the specified boundaries
    scaled_points = []
    for i in range(dim):
        a, b = boundaries[i]
        scaled_dim = a + sobol_points[:, i] * (b - a)
        scaled_points.append(scaled_dim)

    # Transpose the scaled points to get points per dimension
    scaled_points = np.array(list(map(list, zip(*scaled_points))))
    return scaled_points

In [46]:
iter_points = generate_points(dim,n_point,boundaries[:,0],boundaries[:,1])
iter_points

array([[ 0.    ,  0.    ],
       [ 5.    , -5.    ],
       [-5.    ,  5.    ],
       [-2.5   , -2.5   ],
       [ 7.5   ,  7.5   ],
       [ 2.5   , -7.5   ],
       [-7.5   ,  2.5   ],
       [-6.25  , -3.75  ],
       [ 3.75  ,  6.25  ],
       [ 8.75  , -8.75  ],
       [-1.25  ,  1.25  ],
       [-3.75  , -6.25  ],
       [ 6.25  ,  3.75  ],
       [ 1.25  , -1.25  ],
       [-8.75  ,  8.75  ],
       [-8.125 , -0.625 ],
       [ 1.875 ,  9.375 ],
       [ 6.875 , -5.625 ],
       [-3.125 ,  4.375 ],
       [-0.625 , -8.125 ],
       [ 9.375 ,  1.875 ],
       [ 4.375 , -3.125 ],
       [-5.625 ,  6.875 ],
       [-6.875 , -6.875 ],
       [ 3.125 ,  3.125 ],
       [ 8.125 , -1.875 ],
       [-1.875 ,  8.125 ],
       [-4.375 , -4.375 ],
       [ 5.625 ,  5.625 ],
       [ 0.625 , -9.375 ],
       [-9.375 ,  0.625 ],
       [-9.0625, -4.6875],
       [ 0.9375,  5.3125],
       [ 5.9375, -9.6875],
       [-4.0625,  0.3125],
       [-1.5625, -7.1875],
       [ 8.4375,  2.8125],
 

In [47]:
def encode_number(number, min_value=-10, max_value=10, num_bits=32):
    # Normalize the number to a value between 0 and 1
    normalized = (number - min_value) / (max_value - min_value)
    # Convert it to an integer representation
    int_representation = int(normalized * (2**num_bits - 1))
    # Convert the integer to binary and pad with zeros
    return [int(x) for x in format(int_representation, '0{}b'.format(num_bits))]

def decode_number(genome:Genome, min_value=-10, max_value=10, num_bits=32):
    # Convert the binary string to an integer
    int_representation = int(''.join(map(str, genome)), 2)
    # Scale down to the normalized value
    normalized = int_representation / (2**num_bits - 1)
    # Denormalize to get the real number
    return min_value + normalized * (max_value - min_value)

def encode_list(number_list,min_value=-10, max_value=10, num_bits=32):
    encoded = []
    for number in number_list:
        encoded += encode_number(number,min_value, max_value, num_bits)
    return encoded

def decode_list(encoded_list,min_value=-10, max_value=10, num_bits=32):
    numbers = []
    for i in range(0, len(encoded_list), num_bits):
        binary_list = encoded_list[i:i + num_bits]
        number = decode_number(binary_list, min_value, max_value, num_bits)
        numbers.append(number)
    return numbers


In [48]:
def generate_population(set_of_points:np.ndarray,min_value=-10, max_value=10, num_bits=32) -> Population:
    return [encode_list(set_of_points[point],min_value, max_value, num_bits) for point in range(len(set_of_points))]

def single_point_crossover(a: Genome, b: Genome, print_cutoff=False) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")
    length = len(a)
    # if the length less than 2, then there is no point to do the function
    if length < 2:  
        return a, b
    # generate random number as the cutoff of the crossover
    p = randint(1, length - 1)
    if print_cutoff == True:
        print(p)
    
    return a[0:p] + b[p:], b[0:p] + a[p:]

def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    # num: generate how many chromosome(s) that we want to mutate
    for _ in range(num):
        # index sets which chromosome we want to change
        index = randrange(len(genome))
        # the change algorithm
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome

def fitness_function(genome: Genome, min_value=-10, max_value=10, num_bits=32) -> float:
    X = decode_list(genome, min_value, max_value, num_bits)
    return objective_function(X)

# for convenience, call fitness function from these functions below using partial(...)
def population_fitness(population: Population, fitness_func: FitnessFunc) -> float:
    return sum([fitness_func(genome) for genome in population])

def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return choices(
        population=population,
        weights=[fitness_func(gene) for gene in population],
        k=2
    )
def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
    return sorted(population,key= lambda x: fitness_func(x), reverse=True)

def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))

def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc,binary_mode=False):
    print("GENERATION %02d" % generation_id)
    print("=============")
    if binary_mode == True:
        print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
        print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
        sorted_population = sort_population(population, fitness_func)
        print(
            "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
        print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                                fitness_func(sorted_population[-1])))
        print("")
    else:
        print(f"Population: {[decode_list(population[i],num_bits=num_bits) for i in range (len(population))]}")
        print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
        sorted_population = sort_population(population, fitness_func)
        print(f"Best: {(decode_list(population[0],num_bits=num_bits))}")
        print(f"Worst: {(decode_list(population[1],num_bits=num_bits))}")
        print("")
    # return sorted_population[0]

def run_evolution(
        populate_func,
        fitness_func,
        fitness_limit: int,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        sort_func=sort_population,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    population = populate_func()

    for i in range(generation_limit):
        population = sort_func(population,fitness_func)

        if printer is not None:
            printer(population, i, fitness_func)

        # cutoff_criteria = 1-fitness_func(population[0])
        cutoff_criteria = fitness_func(population[0])

        if cutoff_criteria < fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i

In [49]:
popA = generate_population(iter_points,num_bits=num_bits)
print(popA[0])
print(popA[1])
decoded_genome = [decode_list(popA[i],num_bits=num_bits) for i in range (len(popA))]
print(decoded_genome)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[0.0, 0.0], [5.0, -5.0], [-5.0, 5.0], [-2.5, -2.5], [7.5, 7.5], [2.5, -7.5], [-7.5, 2.5], [-6.25, -3.75], [3.75, 6.25], [8.75, -8.75], [-1.25, 1.25], [-3.75, -6.25], [6.25, 3.75], [1.25, -1.25], [-8.75, 8.75], [-8.125, -0.625], [

In [50]:
print(selection_pair(popA,fitness_func=fitness_function))

[[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [51]:
print(selection_pair(population=popA,fitness_func=partial(
    fitness_function,num_bits=num_bits
)))

[[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [52]:
sorted_population2 = sort_population(popA, fitness_func=partial(
    fitness_function,num_bits=num_bits
))

for individual in sorted_population2:
    print(fitness_function(individual, num_bits=num_bits))

print(sorted_population2)


0.39627638912768826
0.3333333333333333
0.2505485680367075
0.23582309207458207
0.10880147963797208
0.0683812105930594
0.040239202890759225
0.035469198320249876
0.02721985362665696
0.02511306306576108
0.018614400942792
0.013082531576073203
0.010373319004390952
0.005303540863990791
0.004308117292534795
0.003218464718849179
0.0029980393323938512
0.0028309547728156735
0.0027043904248523465
0.0024642603242318126
0.0018128506729792632
0.0018123113007010219
0.0017768492338671308
0.0017734207026980308
0.0015999998837761885
0.001187474213304218
0.00102879092057758
0.0009961694607148984
0.0009903827786717425
0.0006679368266707367
0.0005536634938505582
0.0005445925037594007
0.0005013289545041784
0.0004973137747260651
0.0004917584002897436
0.0004719766214202986
0.0004471096706534319
0.0003801994417585863
0.00031583882099927614
0.0002818638341098831
0.00023555361314692613
0.00017059558517211387
4.532433350678587e-05
4.492648424286024e-05
4.468717383539044e-05
4.414725338566254e-05
3.706002301807516e

In [53]:
print_stats(population=popA,generation_id=1,fitness_func=partial(
    fitness_function,num_bits=num_bits
),binary_mode=False
)

GENERATION 01
Population: [[0.0, 0.0], [5.0, -5.0], [-5.0, 5.0], [-2.5, -2.5], [7.5, 7.5], [2.5, -7.5], [-7.5, 2.5], [-6.25, -3.75], [3.75, 6.25], [8.75, -8.75], [-1.25, 1.25], [-3.75, -6.25], [6.25, 3.75], [1.25, -1.25], [-8.75, 8.75], [-8.125, -0.625], [1.875, 9.375], [6.875, -5.625], [-3.125, 4.375], [-0.625, -8.125], [9.375, 1.875], [4.375, -3.125], [-5.625, 6.875], [-6.875, -6.875], [3.125, 3.125], [8.125, -1.875], [-1.875, 8.125], [-4.375, -4.375], [5.625, 5.625], [0.625, -9.375], [-9.375, 0.625], [-9.0625, -4.6875], [0.9375, 5.3125], [5.9375, -9.6875], [-4.0625, 0.3125], [-1.5625, -7.1875], [8.4375, 2.8125], [3.4375, -2.1875], [-6.5625, 7.8125], [-5.3125, -8.4375], [4.6875, 1.5625], [9.6875, -3.4375], [-0.3125, 6.5625], [-2.8125, -0.9375], [7.1875, 9.0625], [2.1875, -5.9375], [-7.8125, 4.0625], [-8.4375, -5.3125], [1.5625, 4.6875], [6.5625, -0.3125]]
Avg. Fitness: 0.032117
Best: [0.0, 0.0]
Worst: [5.0, -5.0]



In [54]:
population,generation = run_evolution(
    populate_func=partial(
        generate_population,set_of_points = iter_points,num_bits=num_bits
    ),
    fitness_func= partial(
        fitness_function, num_bits=num_bits
    ),
    fitness_limit=epsilon,
    generation_limit=k_max
)

In [55]:
len(population)

50

In [56]:
decode_list(population[0],num_bits=num_bits)

[0.6668815203426419, 0.6903458846977788]

In [57]:
for i in range (len(population)):
    print(decode_list(population[i],num_bits=num_bits))

[0.6668815203426419, 0.6903458846977788]
[0.666881520635954, 0.6903458475886772]
[0.8231315203426419, 0.770913408680693]
[0.6668663164970283, 0.692788408680693]
[0.6668743677852778, 0.6903195841485559]
[0.6668719697023988, 0.690345809173623]
[0.6668719696299323, 0.6903673423684982]
[0.6668719696299306, 0.6903458103378117]
[0.666874367785276, 0.6711197733699272]
[0.6717643374992548, 0.6903672668457634]
[0.6677818624308856, 0.6903458103377762]
[0.6668715318716458, 0.6708145975886808]
[0.666881520635954, 0.690310096299827]
[0.6670353049827753, 0.6903088930019567]
[0.8231163165016113, 0.69036249949278]
[0.6717643374946718, 0.766030596180693]
[0.666866316497071, 0.6708145578636522]
[0.6668719697026901, 0.6903625369617359]
[0.6668743677852689, 0.6903648766916639]
[0.6717643374992548, 0.6903196390092976]
[0.6669578145812665, 0.6903458846977788]
[0.6717643328426419, 0.7001114725886808]
[0.6668815203426419, 0.692788408680693]
[0.6668815203429155, 1.0028458091716193]
[0.6668815063730946, 0.67081